# Helm 201

A deep-dive into Helm (v3) and details like

* Templating
* Charts and Subcharts
* Usage and internal structure in Kubernetes
* Integrations

In [ ]:
wd_init = "work/helm-init2"



In [ ]:
!helm version


---
---

## Init

* Create a new template / Helm Chart

In [ ]:
!echo $wd_init
!mkdir -p $wd_init

!helm create $wd_init/demo-helm-201

!tree $wd_init

## Chart Structure and Overview

* `Chart.yaml` common meta information
  * avoid using `appVersion` - handle version in env-specific value-files or as field which will be overwritten during helm execution

In [ ]:
!cat $wd_init/demo-helm-201/Chart.yaml | grep -B2 -i 'version:'

----
* Templating
  * include output/result from namespaced functions
  * Whitespaces and new lines, indent


* Action (00)
  * modify service template

In [ ]:
!echo "Render template and generate Kubernetes resource files"

!helm template demo-helm-201-common $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.yaml --output-dir=work/out/common

----
* Templating, Variables
  * `.Values.*` holds all variables (from file and command line)
  * `values.yaml` and any additional values-file will be merged

 


* Action (01)
  * `image.version`
  * env-specific values file


In [ ]:
!echo "Render template and generate Kubernetes resource files for *TEST* stage"

!helm template demo-helm-201-test $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.test.yaml --output-dir=work/out/test

---

* Templating, Functions
  * `_helpers.tpl` holds set of helper functions used in template files
  * common Go template functions are included (and, or, len, ...)

----
* Templating, Functions
  * `with` set the scope
  * `range` iterate

* Action (02)
  * new `sa.yaml`
  * values in `values.test.yaml`

In [ ]:
!echo "Render template and generate Kubernetes resource files for *TEST* stage"

!helm template demo-helm-201-test $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.test.yaml --output-dir=work/out/test

----
* Charts and Subcharts
  * Subchart is a standalone chart
  * only parent knows the subcharts / childrens
  * ...and only parent can override fields

* Action 
  * checkout existing chart
  * create a new chart in `parent/chart` directory
  * configure dependency

In [ ]:
!echo "Print out existing chart (without subchart)"

!tree $wd_init

In [ ]:
!echo "Create new (sub)chart in the *charts* dir"

!helm create $wd_init/demo-helm-201/charts/demo-subchart
!rm -rf $wd_init/demo-helm-201/charts/demo-subchart/templates/*

!tree $wd_init

* Action (03)
  * template for `ConfigMap` in subchart
  * value file in subchart
  * override options

In [ ]:
!echo "Dry-Run subchart - with local value file"

!helm install --generate-name --dry-run --debug $wd_init/demo-helm-201/charts/demo-subchart

In [ ]:
!echo "Render entire chart - with *pre* value file, to override subchart fields"

!helm template demo-helm-201-pre $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.pre.yaml --output-dir=work/out/pre

In [ ]:
!echo "Render entire chart - with *test* value file, to *NOT* override subchart fields"

!helm template demo-helm-201-test $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.test.yaml --output-dir=work/out/test

---
---

## Life Cycle and Hooks

Hooks allows to execute specific resource definitions at defined points in the Helm life cycle.
Available hooks

* `pre-install`
* `post-install`
* `pre-delete`
* `post-delete`
* `pre-upgrade`
* `post-upgrade`
* `pre-rollback`
* `post-rollback`
* `test`

Examples

* prepare the installation and create specific resources beforehand (ConfigMap, Job etc)
* clean up database before uninstalling application
* return a license or deregister/unsubscribe


### Pre-Install and Post-Delete

Example for `pre-install` and `post-delete` hook. Hooks are usual Kubernetes resource definitions with special annotations.

```
metadata:
  annotations:
    "helm.sh/hook": pre-install, post-delete
    "helm.sh/hook-weight": "-5"
    "helm.sh/hook-delete-policy": hook-succeeded
```

In [ ]:
!echo "Create new (sub)chart in the *charts* dir"

!helm create $wd_init/demo-helm-201/charts/demo-hook
!rm -rf $wd_init/demo-helm-201/charts/demo-hook/templates/ingress.yaml
!rm -rf $wd_init/demo-helm-201/charts/demo-hook/templates/service.yaml
!rm -rf $wd_init/demo-helm-201/charts/demo-hook/templates/tests


!tree $wd_init

---

* Action (04)
  * create 2 jobs for different hooks
  * install and upgrade

In [ ]:
!echo "Install subchart for hook testing"

!echo "some OCP permissions adjustments (for nginx)..."
!oc adm policy add-scc-to-user anyuid -z demo-hook-dev

!helm upgrade --install demo-hook-dev $wd_init/demo-helm-201/charts/demo-hook -n demo-helm

In [ ]:
!echo "Update subchart for hook testing now with job definitions"

!helm upgrade --install demo-hook-dev $wd_init/demo-helm-201/charts/demo-hook -n demo-helm

---

* Action
  * current `pods` and `jobs`
  * events
  * clean-up and check out the job with `post-delete`

In [ ]:
!echo "Delete subchart which triggers hook"

!helm delete demo-hook-dev -n demo-helm

---
---

## OCI Registry

Helm 3 provides the option to store helm charts not only on a HTTP server, but also in a OCI Registry. This allows the option to hold the artifacts (container image and helm charts) in the same registry.

### Testing

* Action
  * use a local registry
  * push a local helm chart
  * install from OCI registry

  

In [ ]:
!echo "Run a local registry"

!docker run -dp 5000:5000 --restart=always --name registry registry

In [ ]:
!echo "Create helm package from the hook subchart"

!echo "...copy subchart in own dir for testing..."
!cp -r $wd_init/demo-helm-201/charts/demo-hook $wd_init/

!echo "...helm package..."
!helm package $wd_init/demo-hook --version 1.0.3 --destination $wd_init
!helm package $wd_init/demo-hook --version 1.0.4 --destination $wd_init

In [ ]:
!echo "Push helm package to registry"

!echo "...helm registry login..."
!helm registry login -u testuser -p testpassword localhost:5000

!echo "...helm package push..."
!helm push $wd_init/demo-hook-1.0.3.tgz oci://localhost:5000/helm-charts
!helm push $wd_init/demo-hook-1.0.4.tgz oci://localhost:5000/helm-charts

In [ ]:
!echo "Verify helm package in OCI registry"

!echo "...show chart details..."
!helm show chart oci://localhost:5000/helm-charts/demo-hook


In [ ]:
!echo "Install (render) helm package directly from OCI registry"

!echo "...render existing chart from registry..."
!helm template release-demo-hook-oci oci://localhost:5000/helm-charts/demo-hook --version 1.0.3 --output-dir=work/out/oci

!echo "...try to render non-existing chart from registry..."
!helm template release-demo-hook-oci oci://localhost:5000/helm-charts/demo-hook --version 2.2.2 --output-dir=work/out/oci


---
---

## Misc

Some additional details


### Internal/Release State

With Helm3 not Tiller exists anymore. All configuration and release state, including value files, is now stored in a `Secret` instead of a `ConfigMap`.


In [ ]:
!echo "Install subchart"

!helm upgrade --install release-demo-hook-state $wd_init/demo-helm-201/charts/demo-hook -n demo-helm

In [ ]:
!echo "List all existing helm releases"

!helm list

In [ ]:
!echo "Print out release information from Helm Release Secret"

!oc get secret sh.helm.release.v1.release-demo-hook-state.v1 -o jsonpath='{.data.release}' | base64 --decode | base64 --decode | gunzip -c | jq

### Helm Upgrade and force re-deployment

Helm determines the relevant changes and updates only the necessary resources. In case the content of a `ConfigMap` changed, but not the image version, a redeployment or restart of the deployment will be not triggered.
To trigger a redeployment a relation between `ConfigMap` content and `Deployment` is needed

```
kind: Deployment
spec:
  template:
    metadata:
      annotations:
        checksum/config: {{ include (print $.Template.BasePath "/configmap.yaml") . | sha256sum }}
```

The idea is, that the `Deployment` contains an annotation with a checksum of the related `ConfigMap`. Any changes in the content, results in an other `sha256sum`, results in an other annotation, which results in modified `Deployment` definition.

### IBM Cloud Container Registry 

The IBM Cloud Container Registry is OCI compliant and could also be used for Helm packages


In [ ]:
!echo "Push chart to IBM CR"

!ibmcloud login --sso

!ibmcloud cr login 

!echo "...use API key to login..."
!helm registry login -u iamapikey

!helm push $wd_init/demo-hook-1.0.1.tgz oci://de.icr.io/demo-helm

### VS Code Extensions

* `yaml`
* `kubernetes` understands also the helm template syntax

## Summary

This was a walkthrough for Helm 201 with covering topics like

* templating
* charts and subcharts
* OCI Registry

## References

* [Helm - Storage Provider - Secret instead ConfigMap](https://helm.sh/docs/topics/advanced/#storage-backends)
* [Helm - (OCI) Registry](https://helm.sh/docs/topics/registries/)
* [Helm - Best Practices](https://helm.sh/docs/chart_best_practices/conventions/)
* [Helm - Template Guide](https://helm.sh/docs/chart_template_guide/getting_started/)